## The Ground Truth Duplicate Problem

Since the test set has overlapping patches from the same tile, and ground truth labels are **tile-based** (not patch-based), then:

### **Scenario: A rock at the boundary between two patches**

```
Tile 2626_1102 (original 2000×2000)
┌─────────────────────────────────┐
│                                 │
│  ┌──────────┐                  │
│  │ Patch    │ 🪨 ← Rock here   │
│  │ 0_0      │                  │
│  └──────────┘                  │
│       ┌──────────┐             │
│       │ Patch    │ 🪨 ← Same rock
│       │ 0_1      │             │
│       └──────────┘             │
│                                 │
└─────────────────────────────────┘
```

**When Alexis created patch labels from tile labels:**
- He took tile-level annotations (point coordinates)
- Generated bounding boxes
- **Assigned the same box to BOTH patches** if rock is in overlap region

**Result:** Ground truth labels have duplicates!

---

## We Need to Remove GT Duplicates

**We will use IoU=0.9**
- Ground truth boxes in overlap regions should be **identical** (same global coordinates)
- IoU=0.9 ensures we only remove exact duplicates, not nearby distinct rocks
- Lower threshold (0.5) might incorrectly merge two real rocks that are close together


## Evaluation Strategy

### **Step 1: Create Clean Ground Truth**

In [1]:
# Remove exact duplicates from ground truth
!python ../scripts/postprocessing/yolo_to_shapefile.py \
  --tif_dir ../data/swisstopo_data/images/test \
  --labels_dir ../data/swisstopo_data/labels/test \
  --out ../outputs/shapefiles/test_GT_with_duplicates.shp \
  --nms_iou 0.0

!python ../scripts/postprocessing/yolo_to_shapefile.py \
  --tif_dir ../data/swisstopo_data/images/test \
  --labels_dir ../data/swisstopo_data/labels/test \
  --out ../outputs/shapefiles/test_GT_clean.shp \
  --nms_iou 0.95  # Very high - only exact duplicates

/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Total detections before NMS: 369
✅ Wrote 369 boxes to ../outputs/shapefiles/test_GT_with_duplicates.shp
CRS: EPSG:2056
/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Total detections before NMS: 369
✂️  Detections after NMS (IoU=0.95): 361
🗑️  Removed duplicates: 8
✅ Wrote 361 boxes to ../outputs/shapefiles/test_GT_clean.shp
CRS: EPSG:2056


In [1]:
!ls

duplicates_suppression.ipynb


In [2]:
%cd ..

/home/donia/Switzerland_Expansion/switzerland_nationwide_rock_detection


/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!python scripts/analysis/visualize_duplicate_pairs.py

VISUALIZING DUPLICATE PAIRS

Loading outputs/shapefiles/test_GT_with_duplicates.shp...
Total boxes: 369

Finding duplicates (IoU ≥ 0.9)...
Found 9 duplicate pairs

Duplicate pairs found:
  Pair 1: 2587_1133_0_3.tif ↔ 2626_1102_3_0.tif (IoU=1.0000)
  Pair 2: 2587_1133_0_3.tif ↔ 2587_1133_3_3.tif (IoU=1.0000)
  Pair 3: 2587_1133_2_2.tif ↔ 2626_1102_3_0.tif (IoU=1.0000)
  Pair 4: 2587_1133_3_2.tif ↔ 2781_1140_1_2.tif (IoU=1.0000)
  Pair 5: 2611_1139_0_1.tif ↔ 2626_1102_3_0.tif (IoU=1.0000)
  Pair 6: 2626_1102_2_0.tif ↔ 2626_1102_2_0.tif (IoU=1.0000)
  Pair 7: 2626_1102_2_0.tif ↔ 2781_1140_0_1.tif (IoU=1.0000)
  Pair 8: 2626_1102_2_0.tif ↔ 2781_1140_0_1.tif (IoU=1.0000)
  Pair 9: 2626_1102_3_0.tif ↔ 2626_1102_3_0.tif (IoU=1.0000)

--- Processing Pair 1/9 ---
  Loading 2587_1133_0_3.tif...
/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  da